In [1]:
import socket
import ssl
import os
import tkinter as tk
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import utils
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


In [2]:

def generate_rsa_keypair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    return private_key

def generate_dh_keypair():
    parameters = dh.generate_parameters(generator=2, key_size=2048)
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key

def derive_dh_shared_key(private_key, public_key):
    shared_key = private_key.exchange(public_key)
    return shared_key

# def derive_dh_shared_key(private_key, public_key):
#     shared_key = public_key.public_numbers().y ** private_key.private_numbers().x % public_key.public_numbers().parameter_numbers.g
#     return shared_key.to_bytes((shared_key.bit_length() + 7) // 8, byteorder='big')


def rsa_encrypt(public_key, plaintext):
    ciphertext = public_key.encrypt(
        plaintext.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return ciphertext

def rsa_decrypt(private_key, ciphertext):
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return plaintext.decode()

def aes_encrypt(key, plaintext):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()
    return iv + ciphertext

def aes_decrypt(key, ciphertext):
    iv = ciphertext[:16]
    ciphertext = ciphertext[16:]
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext.decode()

def start_server(encryption_choice,msg_for_client, append_log, append_chat):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 8443))
    server_socket.listen(1)

    append_log("Server listening on port 8443...")


    client_socket, client_address = server_socket.accept()
    append_log(f"Connection from {client_address} has been established!")

    context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
    append_log("SSL/TLS context created.")
    context.load_cert_chain(certfile="server.crt", keyfile="server.key")
    append_log("SSL/TLS context loaded.")
    
    with context.wrap_socket(client_socket, server_side=True) as ssock:
        append_log("SSL/TLS connection established.")
        
        
        ssock.send(encryption_choice.encode())
        
        if encryption_choice == "1":
            pb_key = ssock.recv(1024)
            append_log(f"Key received from client: {pb_key.decode()}") 
            
            private_key = generate_rsa_keypair()
            public_key = private_key.public_key()
            
            #send public key to client
            ssock.send(public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ))

            client_public_key = load_pem_public_key(pb_key, backend=default_backend())
            append_log(f"Client public key: {client_public_key}")
                
                

            msg_for_client = rsa_encrypt(client_public_key, msg_for_client)
            ssock.send(msg_for_client)
            
            msg_from_client = ssock.recv(1024)
            append_log(f"Encrypted Message from client: {msg_from_client}")
            msg_from_client = rsa_decrypt(private_key, msg_from_client)
            append_chat(f"Decrypted message from client: {msg_from_client}")

        elif encryption_choice == "2":
            # AES encryption and Diffie-Hellman key exchange
            received_public_key = ssock.recv(2048)
            append_log(f"Received public key: {received_public_key}")
            received_public_key = serialization.load_pem_public_key(received_public_key, backend=default_backend())
            
            private_key, public_key = generate_dh_keypair()
            
            ssock.send(public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ))
            
            append_log(f"Public key sent: {public_key}")
            append_log(f"Private key: {private_key}")
            
            try:
                shared_key = derive_dh_shared_key(private_key, received_public_key)
            except:
                append_log("Error: Shared key could not be derived.")
                return
            
            append_log(f"Shared key: {shared_key}")
            msg_from_client = ssock.recv(2048)
            msg_from_client = aes_decrypt(shared_key, msg_from_client)
            append_log(f"Message from client: {msg_from_client}")

            msg_for_client = input("Enter message for client: ")
            msg_for_client = aes_encrypt(shared_key, msg_for_client)
            ssock.send(msg_for_client)




In [3]:
import tkinter as tk
from tkinter import Text, Scrollbar

def append_log(message):
    log_text.config(state=tk.NORMAL)  # Set text widget to allow editing
    log_text.insert(tk.END, message + "\n")
    log_text.config(state=tk.DISABLED)  # Disable editing to make it read-only

def append_chat(message):
    chat_text.config(state=tk.NORMAL)  # Set text widget to allow editing
    chat_text.insert(tk.END, message + "\n")
    chat_text.config(state=tk.DISABLED)  # Disable editing to make it read-only

    
def on_send_message():
    message = message_entry.get()
    encryption_method = encryption_choice.get()

    if encryption_method == "Symmetric":
        start_server("2", message, append_log,  append_chat)
        
        
    elif encryption_method == "Asymmetric":
        start_server("1", message, append_log, append_chat)
        


# Create the main window
root = tk.Tk()
root.title("SERVER")

# Encryption method selection
encryption_choice = tk.StringVar()
encryption_choice.set("Asymmetric")

encryption_label = tk.Label(root, text="Select Encryption Method:")
encryption_label.pack()

asymmetric_radio = tk.Radiobutton(root, text="Asymmetric", variable=encryption_choice, value="Asymmetric")
asymmetric_radio.pack()
symmetric_radio = tk.Radiobutton(root, text="Symmetric", variable=encryption_choice, value="Symmetric")
symmetric_radio.pack()


# Message input
message_label = tk.Label(root, text="Enter Message:")
message_label.pack()

message_entry = tk.Entry(root)
message_entry.pack()

# Send button
send_button = tk.Button(root, text="Send Message", command=on_send_message)
send_button.pack()

# Log display area
log_label = tk.Label(root, text="Log:")
log_label.pack()

log_text = Text(root, height=15, width=100)
log_text.pack()

scrollbar = Scrollbar(root)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

log_text.config(yscrollcommand=scrollbar.set)
scrollbar.config(command=log_text.yview)

chat_label = tk.Label(root, text="Chat:")
chat_label.pack()

chat_text = Text(root, height=5, width=100)
chat_text.pack()

# Start the GUI event loop
root.mainloop()


In [4]:
if __name__ == "__main__":
    start_server("1","hello")


TypeError: start_server() missing 2 required positional arguments: 'append_log' and 'append_chat'

In [ ]:
# # Generate a private key for the server
# openssl genpkey -algorithm RSA -out server.key

# # Create a Certificate Signing Request (CSR) for the server
# openssl req -new -key server.key -out server.csr

# # Sign the server CSR with the CA certificate
# openssl x509 -req -in server.csr -CA ca.crt -CAkey ca.key -CAcreateserial -out server.crt 